In [1]:
import sklearn
import pandas as pd 
import numpy as np
import time

In [2]:
#get data
test_set = pd.read_csv("testing_set.csv")
feature_matrix = pd.read_csv("feature_matrix.csv")

In [3]:
#get user_id from testing set
temp = test_set.columns.values[1:]
test_user_id = []
for i in range(len(temp)):
    test_user_id.append(temp[i])
for i in range(len(test_user_id)):
    test_user_id[i] = int(test_user_id[i])
print("ID of users in the testing set is:")
print(test_user_id)

ID of users in the testing set is:
[37015, 37020, 37032, 37034, 37040, 37049, 37052, 37074, 37097, 37100, 37107, 37110, 37117, 37133, 37149, 37163, 37196, 37199, 37219, 37229, 37233, 37236, 37237, 37264, 37267, 37270, 37326, 37327, 37345, 37372, 37383, 37386, 37399, 37405, 37411, 37423, 37425, 37454, 37459, 37464, 37490, 37494, 37502, 37508, 37520, 37524, 37549, 37558, 37576, 37599, 37609, 37621, 37641, 37662, 37674, 37698, 37702, 37720, 37725, 37754, 37781, 37800, 37803, 37806, 37808, 37823, 37873, 37874, 37903, 37916, 37922, 37928, 37965, 37971, 37979, 37985, 37998, 38004, 38009, 38042, 38045, 38079, 38083, 38087, 38100, 38120, 38137, 38152, 38162, 38213, 38233, 38237, 38253, 38263, 38275, 38278, 38290, 38312, 38318, 38322, 38324, 38367, 38381, 38383, 38388, 38433, 38436, 38462, 38467, 38471, 38487, 38494, 38515, 38521, 38525, 38531, 38537, 38541, 38544, 38555, 38557, 38582, 38586, 38599, 38622, 38644, 38732, 38747, 38753, 38757, 38769, 38777, 38790, 38797, 38803, 38813, 38820, 38833

In [4]:
def get_SubX_and_SubY(n:int):
    #user n
    column_n = str(n) 
    sub_y = test_set[test_set[column_n]!=0][['anime_id',column_n]]
    merge_table = pd.merge(feature_matrix,sub_y,on =  'anime_id') 
    subY = merge_table[column_n].values
    subX = merge_table['genre_vector'].values
    subX = np.array([eval(vector) for vector in subX])
    del sub_y,merge_table
    total,N = 0,0
    for rating in subY:
        if rating != -1:
            total += rating
            N+=1
    if N!=0:
        for i in range(len(subY)):
            if subY[i] == -1:
                subY[i] = round(total/N,2)
    else:
        for i in range(len(subY)):
            subY[i] = 5
    row,col = subX.shape
    #since subX and subY has the same number of row
    #70% and 30% remaining of subX
    subX1,subX2 = subX[:(row*7)//10],subX[-(row*3)//10::]
    #70% and 30% remaining of subY
    subY1,subY2 = subY[:(row*7)//10],subY[-(row*3)//10::]
    del subX,total,N,subY
    return subX1,subX2,subY1,subY2

_,__,___,____ = get_SubX_and_SubY(37020)
print(_,__,___,____)

[[1 1 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]] [[1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] [10.    8.   10.   10.    7.   10.   10.   10.   10.   10.   10.   10.
 10.    8.    6.    7.    8.43 10.   10.   10.   10.    8.   10.    9.
  8.   10.    8.    4.   10.    9.    7.   10.   10.   10.   10.    7.
  6.    8.    8.   10.   10.   10.    7.   10.    6.   10.   10.   10.
 10.    6.    6.    9.    7.    8.43  7.   10.   10.    8.   10.    8.
  9.    7.    8.    7.    6.    8.   10.   10.    6.   10.    7.    8.
  6.    9.    8.    7.    7.    8.   10.   10.    6.    8.    9.    8.
 10.    9.    7.    8.    8.   10.    8.   10.    8.    9.    9.    9.
  7.   10.   10.  ] [ 7.   10.    7.    7.    8.43  9.    8.   10.    9.   10.    9.   10.
  9.    9.    7.    9.    8.    7.    5.    5.    8.43  8.    8.    7.
 10.    7.   10.    8.    8.  

In [5]:
#import model
from sklearn.neural_network import MLPRegressor
def NeuralNetwork():
    model = MLPRegressor(hidden_layer_sizes=(100,100,100,100,100,100,100,100,100,100,100,100,100),activation="relu",solver="sgd",random_state=1,max_iter=10000)
    return model
# hidden_layer_sizes is a tuple (a,b,c,.....)
# a mean the first hidden layer has a neurons, and so on
# the number of elements in the tuple represent the number of hidden layers of the neural network

In [6]:
#helper function
def NeuralNetworkInfo(model):
    print("Information about the neural network")
    print("Hidden layers information:")
    print(model.hidden_layer_sizes)
    print("Activation function: " + model.activation)
    print("Optimization technique using: " + model.solver)
def RMSE(v1 ,v2) -> float:
    
    return np.mean((v1-v2)**2)

In [7]:
print(NeuralNetworkInfo(NeuralNetwork()))

Information about the neural network
Hidden layers information:
(100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100)
Activation function: relu
Optimization technique using: sgd
None


In [8]:
total_rmse = 0
start_time = time.time()
for test_user in test_user_id:
    #model initialization
    model = NeuralNetwork()
    X_train,X_test,y_train,y_test = get_SubX_and_SubY(test_user)
    #train the model
    model.fit(X_train,y_train)
    #predict the ratings after training
    predict_rating = model.predict(X_test)
    #Result
    print("User with ID {}:".format(test_user))
    print("Predict ratings: ")
    print(predict_rating)
    print("True rating: ")
    print(y_test)
    print("RMSE of user with ID {}: ".format(test_user),RMSE(predict_rating,y_test))
    total_rmse += RMSE(predict_rating,y_test)
    print()
end_time = time.time()
print(NeuralNetworkInfo(NeuralNetwork()))
print("RMSE of testing set with 1000 users is: ", total_rmse/len(test_user_id))
print("Executed time: ",end_time - start_time)

User with ID 37015:
Predict ratings: 
[4.88372891 4.8979542 ]
True rating: 
[5. 5.]
RMSE of user with ID 37015:  0.011966155609102727

User with ID 37020:
Predict ratings: 
[8.61625206 8.61986188 8.61968575 8.61436866 8.61506885 8.61539495
 8.61204303 8.61518535 8.61986188 8.61885796 8.61095303 8.61983343
 8.61067103 8.61542057 8.61968575 8.61539495 8.61463538 8.62717069
 8.62298503 8.62454662 8.61968575 8.61095303 8.61885796 8.6066902
 8.61187234 8.61701205 8.61095303 8.61446649 8.61950791 8.61203494
 8.61710288 8.62191334 8.62350635 8.626033   8.60994169 8.6108854
 8.61446649 8.61095303 8.60671503 8.626033   8.626033   8.61442411
 8.62350635]
True rating: 
[ 7.   10.    7.    7.    8.43  9.    8.   10.    9.   10.    9.   10.
  9.    9.    7.    9.    8.    7.    5.    5.    8.43  8.    8.    7.
 10.    7.   10.    8.    8.    9.   10.    8.    7.    9.    8.43  7.
  8.43  3.    7.    7.    8.    6.    8.43]
RMSE of user with ID 37020:  2.5849921561684868

User with ID 37032:
Predict

### Test data: 
1. Hidden layers information:

    (100, 100)

    Activation function: relu

    Optimization technique using: sgd

    RMSE of testing set with 1000 users is:  2.5431703966196473

    Runtime 418s
2. Hidden layers information:

    (100, 100, 100)

    Activation function: relu

    Optimization technique using: sgd

    RMSE of testing set with 1000 users is:  2.2498660590301665

    Executed time:  468.06240010261536s
3. Hidden layers information:

    (100, 100, 100, 100)

    Activation function: relu

    Optimization technique using: sgd

    RMSE of testing set with 1000 users is:  2.1420311954105244
    
    Executed time:  539.9036045074463
4. Hidden layers information:

    (100, 100, 100, 100, 100)
    Activation function: relu

    Optimization technique using: sgd

    RMSE of testing set with 1000 users is:  2.0570181719913303
    
    Executed time:  590.360833644867
5. Hidden layers information:
    
    (100, 100, 100, 100, 100, 100)

    Activation function: relu

    Optimization technique using: sgd

    RMSE of testing set with 1000 users is:  1.9964352234125065
    
    Executed time:  693.3318400382996
6. Hidden layers information:

    (100, 100, 100, 100, 100, 100, 100)

    Activation function: relu

    Optimization technique using: sgd

    RMSE of testing set with 1000 users is:  1.9086170904620738

    Executed time:  839.7897429466248

7. Hidden layers information:

    (100, 100, 100, 100, 100, 100, 100, 100)

    Activation function: relu

    Optimization technique using: sgd

    RMSE of testing set with 1000 users is:  1.9383081664363395
    
    Executed time:  808.4128196239471

8. Hidden layers information:

    (100, 100, 100, 100, 100, 100, 100, 100, 100)

    Activation function: relu

    Optimization technique using: sgd

    RMSE of testing set with 1000 users is:  1.9215414360645455
    
    Executed time:  820.6044371128082
9. Hidden layers information:

    (100, 100, 100, 100, 100, 100, 100, 100, 100, 100)

    Activation function: relu

    Optimization technique using: sgd

    RMSE of testing set with 1000 users is:  1.9067644942132191
    
    Executed time:  812.3373439311981
10. Hidden layers information:

    (100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100)

    Activation function: relu

    Optimization technique using: sgd

    RMSE of testing set with 1000 users is:  1.7986043141628945
    
    Executed time:  821.0979342460632
11. Hidden layers information:

    (100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100)

    Activation function: relu

    Optimization technique using: sgd

    RMSE of testing set with 1000 users is:  1.5243095355147596
    
    Executed time:  559.414577960968